# Escenarios experimentales Internetworking CNN ESCENARIO A
Presentado por: Jonathan Toapanta
Fecha: 23/02/2023


REPOSITORIO : https://github.com/jonathan-elian-toapanta/INTERNETWORKING

In [ ]:
import keras
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np


In [ ]:
# cargar datos y normalizar
url = 'https://raw.githubusercontent.com/jonathan-elian-toapanta/INTERNETWORKING/main/ESCENARIOS/ScenarioA.csv'
df = pd.read_csv(url, low_memory=False)

In [ ]:
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name] = pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value)
    return df

dataframe = df.reindex(np.random.permutation(df.index)).copy()

In [ ]:
keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys)-1]].copy()
x_normalised = dfNormalize(data_to_process)

In [ ]:
y = dataframe['label']
change_labels = lambda x: 1 if x == 'nonTOR' else 0
y_normalised = dataframe['label'].apply(change_labels)


In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_normalised, y_normalised, test_size=0.2, random_state=42)


In [ ]:
# Reshape the data to fit the input shape of the CNN
x_train = x_train.values.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.values.reshape(x_test.shape[0], x_test.shape[1], 1)


In [ ]:
# Convert the labels to binary format
y_train = np.array(y_train).reshape((-1,1))
y_test = np.array(y_test).reshape((-1,1))


In [ ]:
import numpy as np

# Verificar si hay valores nulos en x_train e y_train
print(np.isnan(x_train).any())
print(np.isnan(y_train).any())

# Reemplazar valores nulos por cero
x_train = np.nan_to_num(x_train, nan=0.0)
y_train = np.nan_to_num(y_train, nan=0.0)

In [ ]:
# Definir el modelo
def create_model():
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(x_train.shape[1],1)),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=512, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(units=512, activation='relu'),
        Dropout(0.5),
        Dense(units=1, activation='sigmoid')
        ])
    # Compilar el modelo

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


In [ ]:
# Create the KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=1)


In [ ]:
# Define the grid search parameters
batch_size = [32, 64, 128]
epochs = [10, 15, 20]
param_grid = dict(batch_size=batch_size, epochs=epochs)


In [ ]:
# Perform the grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(x_train, y_train)


In [ ]:
# Print the results
print("Mejor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
print("%f (%f) con: %r" % (mean, stdev, param))